# Data Importing

In [2]:
# Importing modules
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression

# Read data into papers
data = pd.read_csv("/Users/atchoo/Google Drive/Research Assistant/Work With David Knight/Survey of the Incarcerated/TMPPoliticalSurveyFULL_ForDavid.csv", encoding = "ISO-8859-1")

# Print head
# print(data.columns)

In [3]:
# Insert an id index for future work
data["Survey_ID"] = data.index + 1

# Remove the columns
str_data = data[["Survey_ID", "explain_politics_changed_since_incarcerated",\
                 "explain_race_affects_politics", 'identifies_as_black', 'identifies_as_white',
                 "identifies_as_native", 'identifies_as_asian', 
                 'identifies_as_hawaiian_or_pac_islander', 'identifies_as_other_race',
                 'identifies_as_not_sure_of_race', 'identifies_as_hispanic_or_latinx',
                 "age", "gender"]]
# Print out the first rows of papers
str_data.columns = ["Survey_ID", "p_change", "r_effect", "black", "white", "native", "asian",
                   "hawaiian", "other_race", "unsure_race", "latinx", "age", "gender"]
str_data = str_data.dropna()

# Make Remove Stopword Functions

In [4]:
# Load the regular expression library and the nltk word library
import re
import nltk

# Create a function to remove nonsense words
words = set(nltk.corpus.words.words())
def clean_sent(sent):
    return " ".join(w for w in nltk.wordpunct_tokenize(sent) \
     if w.lower() in words or not w.isalpha())

strv_list = ["p_change", "r_effect"]
for strv in strv_list:
    # Remove punctuation & Convert the titles to lowercase
    str_data[strv] = str_data[strv].map(lambda x: re.sub(r'[^\w\s]', "", x)).map(lambda x: x.lower())
    # Remove nonsense words
    str_data[strv] = str_data[strv].apply(clean_sent)
    # Transform blank cells to NaN & Drop NaN
    str_data = str_data.replace(r'^\s*$', np.nan, regex=True).dropna()

In [5]:
str_data.head()

,Survey_ID,p_change,r_effect,black,white,native,asian,hawaiian,other_race,unsure_race,latinx,age,gender
0,1,no,not at all not racist all of the human race is...,False,True,False,False,False,False,False,False,46-55,Man
2,3,i feel that the parole board the some of the p...,not sure about that,False,True,False,False,False,False,False,False,46-55,Woman
3,4,yes bit,i am not really sure about that one,False,True,False,False,False,False,False,False,36-45,Woman
5,6,most are full of,i do not understand this question maybe theres...,True,False,False,False,False,False,False,False,26-35,Woman
6,7,i never considered the prison population or pr...,i try to vote based on the but i am and many i...,False,True,False,False,False,False,False,False,46-55,Woman


In [6]:
# remove stop words
# nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Include the Topic info

In [7]:
def add_the_matrix(extrastopword, csvfile, tlist=["topic 4"]):
    
    assert isinstance(extrastopword, list), "Should be a list of the extra stop words."
    assert isinstance(csvfile, str), "Should be the path of the matrix file."
    
    # change the stop words
    stop_words = stopwords.words("english")
    stop_words.extend(["don", "people", "bill", "step", "act", "first", "u", "n", 
                       "na", "non"]+extrastopword)
    yesnno = ["yes", "no"]
    stop_words =  [word for word in stop_words if word not in yesnno]
    # remove stop words
    r_effect = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in str_data["r_effect"]]
    r_effect = pd.DataFrame([x for x in r_effect if x])
    
    # create index variable for whether a row is left
    r_effect2 = []
    for doc in str_data["r_effect"]:
        if all(word in stop_words for word in simple_preprocess(doc)):
               i_str = False
        else: 
               i_str = True
        r_effect2.append(i_str)
    inBTM = pd.DataFrame(r_effect2, columns=["inBTM"])
    # filter str_data based on inBTM result into str_datar
    str_datar = pd.concat([str_data.reset_index(drop=True), inBTM], axis=1)
    str_datar = str_datar[str_datar["inBTM"]==True]
    
    # read in the topicmatrix file
    matrix = pd.read_csv(csvfile, 
                         names=["topic 1", "topic 2", "topic 3", "topic 4", "topic 5", "topic 6"])
    a = matrix.idxmax(axis=1)
    b = matrix.T.apply(lambda x: x.nlargest(2).idxmin())
    c = matrix.T.apply(lambda x: x.nlargest(3).idxmin())
    topic = pd.concat([a, b, c], axis=1, ignore_index=False)

    # concatenate topic matrix and str_datar as str_datar
    str_datar2 = pd.concat([str_datar.reset_index(drop=True), topic], axis=1)
    str_datar2 = str_datar2.rename(columns={0:"fsttopic", 1:"sndtopic", 2:"trdtopic"})

    # CREATE THE CHISQUARE DATAFRAME
    race = ["black", "white", "native", "asian", "hawaiian", "other_race", "unsure_race", "latinx"]
    topic = ["fsttopic", "sndtopic", "trdtopic"]
    chisq = str_datar2.loc[:, ["Survey_ID"]+race+topic].dropna()
        ## clean the race variable a little bit
    chisq.loc[:, race] = chisq.loc[:, race].astype(int)
    chisq["Race"] = chisq.loc[:, race].idxmax(axis=1)
    
    # CREATE THE REGRESSION DATAFRAME
    chisq["fst4"] = chisq["fsttopic"].isin(tlist)
    chisq["snd4"] = chisq["sndtopic"].isin(tlist)
    chisq["trd4"] = chisq["trdtopic"].isin(tlist)
    cor = chisq.drop(["fsttopic", "sndtopic", "trdtopic", "Race"], axis=1)
    cov = str_data[["Survey_ID", "age", "gender"]]
    cov = pd.get_dummies(cov, columns=["age", "gender"])
    lrdata = cor.merge(cov, on="Survey_ID")
    
    return lrdata

In [8]:
common_csv = "~/Google Drive/Research Assistant/Work With David Knight/Survey of the Incarcerated/BTM/BTM_Regression_Exploration/"
extra1 = []
csv1 = "All_3095/topicmatrix.csv"

extra2 = ["violent", "race"]
csv2 = "NoViloentRace_3082/topicmatrix_3082.csv"

extra3 = ["violent", "race", "white", "black"]
csv3 = "NoViloentRaceWhiteBlack_3053/topicmatrix_3053.csv"

In [9]:
# All_3095
lrdata4 = add_the_matrix(extra1, common_csv+csv1)
# NoViloentRace_3082
lrdata5 = add_the_matrix(extra2, common_csv+csv2)
# NoViloentRaceWhiteBlack_3053
# Most Important One
lrdata6 = add_the_matrix(extra3, common_csv+csv3)

### Clear out all the white samples

In [11]:
def clear_some(dataset):
    dataset_post = dataset[dataset["white"]!=1].drop(["white"], axis=1)
    
    return dataset_post

In [12]:
# All_3095
lrdata4 = clear_some(lrdata4)
# NoViloentRace_3082
lrdata5 = clear_some(lrdata5)
# NoViloentRaceWhiteBlack_3053
# Most Important One
lrdata6 = clear_some(lrdata6)

In [13]:
lrdata4.to_csv("~/Desktop/lrdata_nw4.csv")

# Linear Regression

In [13]:
lrdata4.columns

Index(['Survey_ID', 'black', 'native', 'asian', 'hawaiian', 'other_race',
       'unsure_race', 'latinx', 'fst4', 'snd4', 'trd4', 'age_18-25',
       'age_26-35', 'age_36-45', 'age_46-55', 'age_56-65', 'age_66+',
       'age_Under 18', 'gender_Gender non-conforming or non-binary or other',
       'gender_Man', 'gender_Prefer not to say', 'gender_Woman'],
      dtype='object')

In [38]:
lrdata4.shape

(1260, 22)

In [14]:
def reg_on_123(dataset4lr, dropX=[]):
    y1 = dataset4lr["fst4"]
    y2 = dataset4lr["snd4"]
    y3 = dataset4lr["trd4"]

    X = dataset4lr.drop(["Survey_ID", "fst4", "snd4", "trd4", "unsure_race", "age_Under 18",
                         "gender_Gender non-conforming or non-binary or other"]+dropX, axis=1)
    X = sm.add_constant(X)
    
    model1 = sm.OLS(y1, X).fit()
    model2 = sm.OLS(y2, X).fit()
    model3 = sm.OLS(y3, X).fit()
    
    return model1, model2, model3

def return_plist(dataset4lr, dropX=[]):
    model1, model2, model3 = reg_on_123(dataset4lr, dropX)
    
    a = (model1.summary2().tables[1]['P>|t|'] <= 0.05) | (model1.summary2().tables[1]['P>|t|'] >= 0.95)
    a1 = model1.summary2().tables[1]["Coef."]
    b = (model2.summary2().tables[1]['P>|t|'] <= 0.05) | (model2.summary2().tables[1]['P>|t|'] >= 0.95)
    b1 = model2.summary2().tables[1]["Coef."]
    c = (model3.summary2().tables[1]['P>|t|'] <= 0.05) | (model3.summary2().tables[1]['P>|t|'] >= 0.95)
    c1 = model3.summary2().tables[1]["Coef."]
    
    p_df = pd.concat([a1, a, b1, b, c1, c], axis=1)
    p_df.columns = ["fst_inlist_coef", "p<=0.05", "snd_inlist_coef", "p<=0.05", "trd_inlist_coef", "p<=0.05"]
    
    return p_df

### Keep All
These groups of models only excluded some variables to clear perfect multicollinearity.
The variables excluded for perfect multicollinearity include "Survey_ID", "fst1n2", "snd1n2", "trd1n2", "unsure_race","age_Under 18", "gender_Gender non-conforming or non-binary or other".

In [15]:
# All_3095
S3095_all = return_plist(lrdata4)
# NoViloentRace_3082
S3082_all = return_plist(lrdata4)
# NoViloentRaceWhiteBlack_3053
S3053_all = return_plist(lrdata4)

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [16]:
print(S3095_all)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.437180     True         0.184802    False   
black                            0.033523    False        -0.004377    False   
native                          -0.011075    False        -0.029330    False   
asian                            0.085756    False        -0.015011    False   
hawaiian                         0.023533    False        -0.139388    False   
other_race                       0.046690    False        -0.042583    False   
latinx                          -0.075235     True         0.069159     True   
age_18-25                       -0.073784    False         0.136036    False   
age_26-35                       -0.123970    False         0.169233    False   
age_36-45                       -0.144314    False         0.197149    False   
age_46-55                       -0.095236    False         0.155239    False   
age_56-65                       -0.12926

In [17]:
print(S3082_all)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.437180     True         0.184802    False   
black                            0.033523    False        -0.004377    False   
native                          -0.011075    False        -0.029330    False   
asian                            0.085756    False        -0.015011    False   
hawaiian                         0.023533    False        -0.139388    False   
other_race                       0.046690    False        -0.042583    False   
latinx                          -0.075235     True         0.069159     True   
age_18-25                       -0.073784    False         0.136036    False   
age_26-35                       -0.123970    False         0.169233    False   
age_36-45                       -0.144314    False         0.197149    False   
age_46-55                       -0.095236    False         0.155239    False   
age_56-65                       -0.12926

In [18]:
print(S3053_all)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.437180     True         0.184802    False   
black                            0.033523    False        -0.004377    False   
native                          -0.011075    False        -0.029330    False   
asian                            0.085756    False        -0.015011    False   
hawaiian                         0.023533    False        -0.139388    False   
other_race                       0.046690    False        -0.042583    False   
latinx                          -0.075235     True         0.069159     True   
age_18-25                       -0.073784    False         0.136036    False   
age_26-35                       -0.123970    False         0.169233    False   
age_36-45                       -0.144314    False         0.197149    False   
age_46-55                       -0.095236    False         0.155239    False   
age_56-65                       -0.12926

### No Black
Other than same variables above to clear perfect multicollinearity, these groups of models also excluded "black".

In [21]:
nb = ["black"]

In [22]:
# All_3095
S3095_nb = return_plist(lrdata4, nb)
# NoRace_3082
S3082_nb = return_plist(lrdata5, nb)
# NoRaceWhiteBlack_3053
S3053_nb = return_plist(lrdata6, nb)

In [23]:
print(S3095_nb)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.464452     True         0.181241    False   
native                          -0.028624    False        -0.027038    False   
asian                            0.069487    False        -0.012887    False   
hawaiian                         0.007865    False        -0.137343    False   
other_race                       0.027976    False        -0.040139    False   
latinx                          -0.083987     True         0.070302     True   
age_18-25                       -0.068834    False         0.135390    False   
age_26-35                       -0.119660    False         0.168670    False   
age_36-45                       -0.139860    False         0.196567    False   
age_46-55                       -0.090735    False         0.154652    False   
age_56-65                       -0.124693    False         0.147971    False   
age_66+                         -0.12538

In [28]:
print(S3082_nb)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.393740    False         0.038503    False   
native                           0.075226     True        -0.048462    False   
asian                           -0.072721    False        -0.097973    False   
hawaiian                        -0.021502    False         0.007511    False   
other_race                      -0.002334    False        -0.056865     True   
latinx                           0.017643    False         0.001617     True   
age_18-25                       -0.193255    False         0.182342    False   
age_26-35                       -0.130262    False         0.175317    False   
age_36-45                       -0.201310    False         0.170405    False   
age_46-55                       -0.221820    False         0.204223    False   
age_56-65                       -0.187431    False         0.218373    False   
age_66+                         -0.19774

In [40]:
S3053_nb.to_csv("~/Desktop/b.csv")

### Only Black
Other than same variables above to clear perfect multicollinearity, these groups of models also excluded all race binary variables excpet "black".

In [30]:
ob = ["native", "asian", "hawaiian", "other_race", "unsure_race", "latinx"]

In [31]:
# All_3095
S3095_ob = return_plist(lrdata4, ob)
# NoRace_3082
S3082_ob = return_plist(lrdata5, ob)
# NoRaceWhiteBlack_3053
S3053_ob = return_plist(lrdata6, ob)

In [32]:
print(S3095_ob)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.386994     True         0.173184    False   
black                            0.039855     True         0.000884     True   
age_18-25                       -0.038534    False         0.141889    False   
age_26-35                       -0.085595    False         0.169402    False   
age_36-45                       -0.100600    False         0.191049    False   
age_46-55                       -0.053804    False         0.153038    False   
age_56-65                       -0.083884    False         0.143820    False   
age_66+                         -0.077984    False         0.091800    False   
gender_Man                      -0.194965     True        -0.173361    False   
gender_Prefer not to say        -0.283362     True        -0.273444     True   
gender_Woman                    -0.207872     True        -0.191214    False   

                          trd_inlist_co

In [42]:
S3095_ob.to_csv("~/Desktop/b.csv")

In [33]:
print(S3082_ob)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.471711     True        -0.013984    False   
black                           -0.039486    False         0.042092    False   
age_18-25                       -0.215712    False         0.166609    False   
age_26-35                       -0.151482    False         0.158163    False   
age_36-45                       -0.222730    False         0.151952    False   
age_46-55                       -0.241387    False         0.189304    False   
age_56-65                       -0.211487    False         0.204908    False   
age_66+                         -0.219117    False         0.254565    False   
gender_Man                      -0.063814    False         0.005565     True   
gender_Prefer not to say         0.035518    False        -0.091853    False   
gender_Woman                    -0.031257    False        -0.063495    False   

                          trd_inlist_co

In [34]:
print(S3053_ob)

                          fst_inlist_coef  p<=0.05  snd_inlist_coef  p<=0.05  \
const                            0.231046    False        -0.050892    False   
black                           -0.036978    False        -0.054670     True   
age_18-25                       -0.045577    False         0.106684    False   
age_26-35                       -0.071509    False         0.154333    False   
age_36-45                       -0.078297    False         0.202768    False   
age_46-55                        0.000247     True         0.174136    False   
age_56-65                       -0.019092    False         0.204856    False   
age_66+                         -0.103187    False         0.143525    False   
gender_Man                      -0.023650    False         0.061826    False   
gender_Prefer not to say         0.027010    False         0.113632    False   
gender_Woman                    -0.027370    False         0.113505    False   

                          trd_inlist_co

In [41]:
S3053_ob.to_csv("~/Desktop/a.csv")